In [1]:
import shutil
from keras import layers
from keras import models
from tensorflow.keras import optimizers
from palmtools import plotModelStats
from palmtools import datasetCounter
from palmtools import extractFeatures
from palmtools import createDirectoryArch
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG19
from tensorflow.keras import callbacks
from tensorflow.keras.models import load_model
import palmtools
import const
import os
import numpy as np

%cd '/home/leonard/Desktop/PyProjects/palm-cnn/'

/home/leonard/Desktop/PyProjects/palm-cnn


In [2]:
train_ratio = 0.75
valid_ratio = 0.15
test_ratio = 0.10

In [6]:
files = palmtools.getFilesArray(const.GENERATED,const.PALM)
palmtools.createDataset(const.PALM,files,train_ratio, valid_ratio, test_ratio)

In [10]:
files[5245]

['/home/leonard/Desktop/PyProjects/palm-cnn/FYODB/Generated_Images/Session1/Palm/',
 's33_8_R_S2.jpg',
 's1.',
 '33_R',
 33,
 44,
 101]

In [ ]:
train_ratio = 0.8
valid_ratio = 0.2
test_ratio = 0

In [ ]:
files = palmtools.getFilesArray(const.ROI,const.PALM)
palmtools.createDataset(const.PALM,files,train_ratio, valid_ratio, test_ratio)

In [ ]:
conv_base_vgg16 = VGG16(weights='imagenet', include_top=False,
                      input_shape=(150,150,3))

In [ ]:
# inghetarea straturilor pentru a preveni alterarea ponderilor preantrenate
print("Nr. parametrii antrenabili VGG16: ",len(conv_base_vgg16.trainable_weights))

conv_base_vgg16.trainable = False

print("Nr. parametrii antrenabili VGG16: ",len(conv_base_vgg16.trainable_weights))


In [ ]:
vgg16_out_shape = (4, 4, 512)
vgg19_out_shape = (4, 4, 512)
resnet_out_shape= (5, 5, 2048)

batch_size = 32
#rotation_range=15, width_shift_range=0.02, height_shift_range=0.01, shear_range=0.01, zoom_range=[0.9, 1.25], horizontal_flip=False, fill_mode='nearest', brightness_range=[0.3, 1]

In [ ]:

image_gen = ImageDataGenerator(rescale = 1./255)

palm_train_generator = image_gen.flow_from_directory(
    const.PALM_DS_DIR + const.TRAIN_DIR,
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='categorical'
)

palm_valid_generator = image_gen.flow_from_directory(
    const.PALM_DS_DIR + const.VALID_DIR,
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='categorical'
)

palm_test_generator = image_gen.flow_from_directory(
    const.PALM_DS_DIR + const.TEST_DIR,
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
palm_train_features_vgg16, palm_train_labels_vgg16 = extractFeatures(conv_base_vgg16,
                                                                     palm_train_generator,
                                                                     (int(6400*train_ratio),) + vgg16_out_shape,
                                                                     batch_size)

In [ ]:
palm_valid_features_vgg16, palm_valid_labels_vgg16 = extractFeatures(conv_base_vgg16,
                                                                      palm_valid_generator,
                                                                      (int(6400*valid_ratio),) + vgg16_out_shape,
                                                                      batch_size)



In [ ]:
vgg16_clasificator = models.Sequential()
vgg16_clasificator.add(layers.Flatten())
vgg16_clasificator.add(layers.Dense(4096, activation='relu', input_dim=(4 * 4 * 512)))
vgg16_clasificator.add(layers.Dropout(0.5))
vgg16_clasificator.add(layers.Dense(4096, activation='relu'))
vgg16_clasificator.add(layers.Dropout(0.5))
vgg16_clasificator.add(layers.Dense(160, activation='softmax'))
vgg16_clasificator.compile(loss='categorical_crossentropy',
                           metrics=['accuracy'],
                           optimizer=optimizers.RMSprop(learning_rate=1e-5, momentum=0.9))

In [ ]:
filepath = 'Models/Checkpoints/vgg16/Classifier/weights-improvment.hdf5'

vgg16_clasificator_checkpoint = callbacks.ModelCheckpoint(
    filepath=filepath,
#    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1)

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5)

history_vgg16_clasificator = vgg16_clasificator.fit(palm_train_features_vgg16,palm_train_labels_vgg16,
                      epochs=100,
                      batch_size=batch_size,
                      validation_data=(palm_valid_features_vgg16,palm_valid_labels_vgg16),
                                                    callbacks=[early_stopping,vgg16_clasificator_checkpoint])


In [ ]:
import pickle
with open('Models/Checkpoints/vgg16/vgg16_clasificator_history', 'wb') as file_pi:
    pickle.dump(history_vgg16_clasificator.history, file_pi)

In [ ]:
plotModelStats(history_vgg16_clasificator)

In [ ]:
vgg16_clasificator.load_weights('Models/Checkpoints/vgg16/Classifier/weights-improvment.hdf5')

In [ ]:
print("Numarul de parametrii antrenabili: ",len(conv_base_vgg16.trainable_weights))
conv_base_vgg16.trainable = True
set_trainable = False
for layer in conv_base_vgg16.layers:
    if ('block5' in layer.name):
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
print("Numarul de parametrii antrenabili: ",len(conv_base_vgg16.trainable_weights))
conv_base_vgg16.compile(loss='categorical_crossentropy',
                  metrics=['acc'])

In [ ]:
model_complet = models.Sequential()

model_complet.add(conv_base_vgg16)
model_complet.add(vgg16_clasificator)

model_complet.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.RMSprop(learning_rate=1e-5, momentum=0.9),
                      metrics=['accuracy'])

In [ ]:
filepath = 'Models/Checkpoints/vgg16/Model_complet/weights-improvment.hdf5'

vgg16_model_checkpoint = callbacks.ModelCheckpoint(
    filepath=filepath,
#    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1)

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5)


history_complet = model_complet.fit(palm_train_generator,
                      epochs=100,   
                      batch_size=batch_size,
                      validation_data=palm_valid_generator,
                                   callbacks=[early_stopping,vgg16_model_checkpoint])

In [ ]:
plotModelStats(history_complet)

In [ ]:
import pickle
with open('Models/Checkpoints/vgg16/vgg16_model_complet_history', 'wb') as file_pi:
    pickle.dump(history_complet.history, file_pi)

In [ ]:
test_loss, test_acc = model_complet.evaluate(palm_test_generator, steps=20)
print('test acc:', test_acc)

In [ ]:
conv_base_resnet = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(150,150,3))

In [ ]:
# inghetarea straturilor pentru a preveni alterarea ponderilor preantrenate
print("Nr. parametrii antrenabili ResNet50: ",len(conv_base_resnet.trainable_weights))

conv_base_resnet.trainable = False

print("Nr. parametrii antrenabili ResNet50: ",len(conv_base_resnet.trainable_weights))


In [ ]:
palm_train_features_resnet, palm_train_labels_resnet = extractFeatures(conv_base_resnet, 
                                                                       palm_train_generator,
                                                                       (int(6400*train_ratio),) + resnet_out_shape,
                                                                       batch_size)



In [ ]:
palm_valid_features_resnet, palm_valid_labels_resnet = extractFeatures(conv_base_resnet,
                                                                       palm_valid_generator,
                                                                       (int(6400*valid_ratio),) + resnet_out_shape,
                                                                       batch_size)



In [ ]:
np.save('palm_train_features_resnet.bin',palm_train_features_resnet)
np.save('palm_train_labels_resnet.bin',palm_train_labels_resnet)
np.save('palm_valid_features_resnet.bin',palm_valid_features_resnet)
np.save('palm_valid_labels_resnet.bin',palm_valid_labels_resnet)

In [ ]:
palm_train_features_resnet=np.load('palm_train_features_resnet.npy')
palm_train_labels_resnet=np.load('palm_train_labels_resnet.npy')
palm_valid_features_resnet=np.load('palm_valid_features_resnet.npy')
palm_valid_labels_resnet=np.load('palm_valid_labels_resnet.npy')

In [ ]:
resnet_clasificator = models.Sequential()
resnet_clasificator.add(layers.Flatten())
resnet_clasificator.add(layers.Dense(4096, activation='relu', input_dim=(5 * 5 * 2048)))
resnet_clasificator.add(layers.Dropout(0.5))
resnet_clasificator.add(layers.Dense(4096, activation='relu'))
resnet_clasificator.add(layers.Dropout(0.5))
resnet_clasificator.add(layers.Dense(160, activation='softmax'))
resnet_clasificator.compile(loss='categorical_crossentropy',
                            metrics=['accuracy'],
                            )

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5)

filepath = 'Models/Checkpoints/resnet/Classifier/weights-improvment.hdf5'

resnet_clasificator_checkpoint = callbacks.ModelCheckpoint(
    filepath=filepath,
 #   save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

history_resnet_clasificator = resnet_clasificator.fit(palm_train_features_resnet,
                                                      palm_train_labels_resnet,
                                                        epochs=100,
                                                        batch_size=batch_size,
                                                        validation_data=(palm_valid_features_resnet,palm_valid_labels_resnet),
                                                        callbacks=[early_stopping,resnet_clasificator_checkpoint])

In [ ]:
model_complet_resnet = models.Sequential()
model_complet_resnet.add(conv_base_resnet)
model_complet_resnet.add(resnet_clasificator)
model_complet_resnet.compile(loss='categorical_crossentropy',
                            metrics=['accuracy'],
                            optimizer=optimizers.RMSprop(learning_rate=1e-5, momentum=0.9))


In [ ]:
resnet_clasificator.summary()
conv_base_resnet.summary()
model_complet_resnet.summary()

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5)

filepath = 'Models/Checkpoints/resnet/Model_complet/weights-improvment.hdf5'

resnet_model_complet_checkpoint = callbacks.ModelCheckpoint(
    filepath=filepath,
 #   save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

history_resnet_model_complet = model_complet_resnet.fit(palm_train_generator,
                                                        epochs=100,
                                                        batch_size=batch_size,
                                                        validation_data=palm_valid_generator,
                                                        callbacks=[early_stopping,resnet_model_complet_checkpoint])


In [ ]:
conv_base_vgg19 = VGG19(weights='imagenet', include_top=False,
                      input_shape=(150,150,3))

In [ ]:
# inghetarea straturilor pentru a preveni alterarea ponderilor preantrenate
print("Nr. parametrii antrenabili VGG19: ",len(conv_base_vgg19.trainable_weights))

conv_base_vgg19.trainable = False

print("Nr. parametrii antrenabili VGG19: ",len(conv_base_vgg19.trainable_weights))


In [ ]:
conv_base_vgg16.summary()
conv_base_vgg19.summary()
conv_base_resnet.summary()

In [ ]:
palm_train_features_vgg19, palm_train_labels_vgg19 = extractFeatures(conv_base_vgg19,
                                                                      palm_train_generator,
                                                                      (int(6400*train_ratio),) + vgg19_out_shape,
                                                                      batch_size
                                                                     )


In [ ]:
palm_valid_features_vgg19, palm_valid_labels_vgg19 = extractFeatures(conv_base_vgg19,
                                                                      palm_valid_generator,
                                                                      (int(6400*valid_ratio),) + vgg19_out_shape,
                                                                      batch_size
                                                                     )

In [ ]:
vgg19_clasificator = models.Sequential()
vgg19_clasificator.add(layers.Flatten())
vgg19_clasificator.add(layers.Dense(4096, activation='relu', input_dim=(4 * 4 * 512)))
vgg19_clasificator.add(layers.Dropout(0.5))
vgg19_clasificator.add(layers.Dense(4096, activation='relu'))
vgg19_clasificator.add(layers.Dropout(0.5))
vgg19_clasificator.add(layers.Dense(160, activation='softmax'))
vgg19_clasificator.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=optimizers.RMSprop(learning_rate=1e-5, momentum=0.9))

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=6)

vgg16_class_path = 'Models/Checkpoints/vgg16/Classifier/'
vgg19_class_path = 'Models/Checkpoints/vgg19/Classifier/'
resnet_class_path = 'Models/Checkpoints/resnet/Classifier/'

vgg16_model_path = 'Models/Checkpoints/vgg16/Model/'
vgg19_model_path = 'Models/Checkpoints/vgg19/Model/'
resnet_model_path = 'Models/Checkpoints/resnet/Model/'



vgg16_clasificator_checkpoint = callbacks.ModelCheckpoint(
    filepath=vgg16_class_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
vgg16_model_checkpoint = callbacks.ModelCheckpoint(
    filepath=vgg16_model_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)



vgg19_clasificator_checkpoint = callbacks.ModelCheckpoint(
    filepath=vgg19_class_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

vgg19_model_checkpoint = callbacks.ModelCheckpoint(
    filepath=vgg19_model_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)



resnet_clasificator_checkpoint = callbacks.ModelCheckpoint(
    filepath=resnet_class_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

resnet_model_checkpoint = callbacks.ModelCheckpoint(
    filepath=resnet_model_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)